In [ ]:
import pymysql
mydb = pymysql.connect(host="", user="", password="")

In [2]:
mycursor = mydb.cursor()

In [3]:
mycursor.execute("USE mydatabase")

0

In [6]:
mycursor.execute("""CREATE TABLE user_profiles (
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    age INT,
    gender VARCHAR(20),
    marital_status VARCHAR(50),
    education VARCHAR(100),
    job VARCHAR(100),
    location VARCHAR(100),
    
    -- Top interests as separate columns
    first_interest VARCHAR(50),
    first_interest_percentage INT,
    second_interest VARCHAR(50),
    second_interest_percentage INT,
    third_interest VARCHAR(50),
    third_interest_percentage INT,
    
    personality_summary TEXT,
    
    -- Activities (stored as JSON array)
    key_activities TEXT,
    
    total_posts INT,
    
    -- Habits (stored as JSON array)
    top_habits TEXT,
    
    top_hobby VARCHAR(100),
    travel_indicators VARCHAR(50),
    
    -- Life indicators (stored as JSON array)
    life_indicators TEXT,
    
    -- Spending indicators (stored as JSON array)
    spending_indicators TEXT
)""")

0

In [ ]:
from typing import Dict, Any
import pymysql
import json

class Database:
    def __init__(self, host: str, user: str, password: str, db: str):
        """Initialize the database connection"""
        self.connection = pymysql.connect(
            host=host,
            user=user,
            password=password,
            db=db,
            cursorclass=pymysql.cursors.DictCursor
        )
        self.cursor = self.connection.cursor()

    def insert_user_profile(self, profile_data: Dict[str, Any]) -> bool:
        """Insert a user profile into the database"""
        try:
            # Convert lists to JSON strings
            key_activities = json.dumps(profile_data.get("key_activities", []))
            top_habits = json.dumps(profile_data.get("top_habits", []))
            life_indicators = json.dumps(profile_data.get("life_indicators", []))
            spending_indicators = json.dumps(profile_data.get("spending_indicators", []))
            
            # Get interests from the top_interests list
            interests = profile_data.get("top_interests", [{} for _ in range(3)])
            
            sql = """
            INSERT INTO user_profiles (
                first_name, last_name, age, gender, marital_status, education, job, location,
                first_interest, first_interest_percentage,
                second_interest, second_interest_percentage,
                third_interest, third_interest_percentage,
                personality_summary, key_activities, total_posts, top_habits,
                top_hobby, travel_indicators, life_indicators, spending_indicators
            ) VALUES (
                %s, %s, %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s,
                %s, %s, %s, %s, %s, %s, %s, %s
            )
            """

            values = (
                profile_data.get("first_name"),
                profile_data.get("last_name"),
                profile_data.get("age"),
                profile_data.get("gender"),
                profile_data.get("marital_status"),
                profile_data.get("education"),
                profile_data.get("job"),
                profile_data.get("location"),
                interests[0].get("interest"),
                interests[0].get("percentage"),
                interests[1].get("interest"),
                interests[1].get("percentage"),
                interests[2].get("interest"),
                interests[2].get("percentage"),
                profile_data.get("personality_summary"),
                key_activities,
                profile_data.get("total_posts"),
                top_habits,
                profile_data.get("top_hobby"),
                profile_data.get("travel_indicators"),
                life_indicators,
                spending_indicators
            )

            self.cursor.execute(sql, values)
            self.connection.commit()
            self.logger.info(f"Successfully inserted profile for {profile_data.get('first_name')} {profile_data.get('last_name')}")
            return True

        except pymysql.MySQLError as e:
            self.logger.error(f"Error inserting user profile: {e}")
            return False
        except Exception as e:
            self.logger.error(f"Unexpected error: {e}")
            return False
